In [1]:
import numpy as np
from PIL import Image
import pandas as pd
from glob import glob
import shutil

In [2]:
computername = 'PCM'
consensus_grades_severity = pd.read_csv(f'/mnt/c/Users/{computername}/Dropbox/chla_fundus/consensus_grades_severity.csv')
labels = pd.read_csv(f'/mnt/c/Users/{computername}/Dropbox/chla_fundus/labels.csv')
metadata = pd.read_csv(f'/mnt/c/Users/{computername}/Dropbox/chla_fundus/metadata.csv')

## Severity Classification

Create dataset train/test/val in form of pandas dataframe

In [3]:
# get severity labels of 1,2,3,4,5 (less to more severe)
def get_subject_number(path):
    return int(path.split('_')[0])

severity_labels = consensus_grades_severity.drop(['Unnamed: 0','Filename', 'Optos'], axis=1).rename(columns={"original name": "path", "consensus_grade": "labels"})
severity_labels['subject'] = severity_labels['path'].apply(get_subject_number)

# get severity labels of 0 (Healthy)
pseudopap = labels[labels.label==1].drop(['site'], axis=1)
pseudopap['labels'] = pseudopap['label'] * 0
pseudopap = pseudopap.drop(['label'], axis=1)

# concat two above
severity_labels = pd.concat([severity_labels, pseudopap], ignore_index=True)

In [44]:
subs = pd.DataFrame(severity_labels[['subject','labels']].groupby(['subject']).mean().astype(int)).reset_index()
subs.head()

,subject,labels
0,2,0
1,3,4
2,4,0
3,11,0
4,20,0


In [28]:
subjects = subs.subject
avglabel = subs.labels

In [98]:
from sklearn.model_selection import train_test_split
X_trainval, X_test, y_trainval, y_test = train_test_split(subjects, avglabel, test_size=0.1, random_state=42, stratify=avglabel)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, random_state=42, stratify=y_trainval)

In [99]:
severity_labels.loc[severity_labels['subject'].isin(X_train)].head()

,path,labels,subject
0,328_visit01_photo02.jpg,2,328
3,182_visit03_photo01.jpg,3,182
4,225_visit01_photo01.jpg,4,225
6,233_visit02_photo02.jpg,3,233
7,300_visit02_photo02.jpg,3,300


In [95]:
import os
os.system('rm -rf /mnt/c/Users/PCM/Dropbox/chla_fundus_croped/severity')
os.system('mkdir /mnt/c/Users/PCM/Dropbox/chla_fundus_croped/severity')
for phase in ['train', 'test', 'val']:
    os.system(f'mkdir /mnt/c/Users/PCM/Dropbox/chla_fundus_croped/severity/{phase}')
    for i in range(6):
        os.system(f'mkdir /mnt/c/Users/PCM/Dropbox/chla_fundus_croped/severity/{phase}/{i}')


In [ ]:
path2fundus = f'/mnt/c/Users/{computername}/Dropbox/chla_fundus_croped'
count = 0
for phase in ['train', 'test', 'val']:
    
    try:
        shutil.copy(f'{path2fundus}/full/' + severity_labels.path[i], f'{path2fundus}/severity/full/{severity_labels.labels[i]}')
    except:
        count = count + 1
        print(f'No Image {severity_labels.path[i]} name found')
print('Total images not present in the dataset: ', count)

In [102]:
phase = 'train'
data = severity_labels.loc[severity_labels['subject'].isin(eval(f'X_{phase}'))]
data.head()

,path,labels,subject
0,328_visit01_photo02.jpg,2,328
3,182_visit03_photo01.jpg,3,182
4,225_visit01_photo01.jpg,4,225
6,233_visit02_photo02.jpg,3,233
7,300_visit02_photo02.jpg,3,300


In [109]:
path2fundus = f'/mnt/c/Users/{computername}/Dropbox/chla_fundus_croped'
count = 0
for phase in ['train', 'test', 'val']:
    data = severity_labels.loc[severity_labels['subject'].isin(eval(f'X_{phase}'))].path.to_numpy()
    labels = severity_labels.loc[severity_labels['subject'].isin(eval(f'X_{phase}'))].labels.to_numpy()
    for i in range(len(data)):
        try:
            shutil.copy(f'{path2fundus}/full/' + data[i], f'{path2fundus}/severity/{phase}/{labels[i]}')
        except:
            count = count + 1
            print(f'No Image {data[i]} name found')

print('Total images not present in the dataset: ', count)

No Image 203_visit01_photo02.jpg name found
No Image 174_visit03_photo01.jpg name found
No Image 174_visit06_photo02.jpg name found
No Image 334_visit01_photo02.jpg name found
No Image 343_visit02_photo02.jpg name found
No Image 195_visit01_photo01.jpg name found
No Image 329_visit01_photo02.jpg name found
No Image 200_visit03_photo01.jpg name found
No Image 343_visit02_photo01.jpg name found
No Image 196_visit01_photo01.jpg name found
No Image 193_visit01_photo01.jpg name found
No Image 232_visit05_photo01.jpg name found
No Image 44_visit03_photo01.jpg name found
No Image 173_visit03_photo02.jpg name found
No Image 39_visit01_photo01.jpg name found
No Image 178_visit01_photo01.jpg name found
No Image 207_visit02_photo02.jpg name found
No Image 334_visit01_photo01.jpg name found
No Image 233_visit01_photo02.jpg name found
No Image 232_visit03_photo02.jpg name found
No Image 227_visit03_photo01.jpg name found
No Image 227_visit03_photo02.jpg name found
No Image 197_visit01_photo01.jpg n

Create binary class dataset

In [112]:
import os
os.system('rm -rf /mnt/c/Users/PCM/Dropbox/chla_fundus_croped/binary')
os.system('mkdir /mnt/c/Users/PCM/Dropbox/chla_fundus_croped/binary')
for phase in ['train', 'test', 'val']:
    os.system(f'mkdir /mnt/c/Users/PCM/Dropbox/chla_fundus_croped/binary/{phase}')
    for i in range(2):
        os.system(f'mkdir /mnt/c/Users/PCM/Dropbox/chla_fundus_croped/binary/{phase}/{i}')


In [114]:
labels[1]==4

True

In [115]:
path2fundus = f'/mnt/c/Users/{computername}/Dropbox/chla_fundus_croped'
count = 0
for phase in ['train', 'test', 'val']:
    data = severity_labels.loc[severity_labels['subject'].isin(eval(f'X_{phase}'))].path.to_numpy()
    labels = severity_labels.loc[severity_labels['subject'].isin(eval(f'X_{phase}'))].labels.to_numpy()
    for i in range(len(data)):
        try:
            shutil.copy(f'{path2fundus}/full/' + data[i], f'{path2fundus}/binary/{phase}/{(labels[i]!=0)*1}')
        except:
            count = count + 1
            print(f'No Image {data[i]} name found')

print('Total images not present in the dataset: ', count)

No Image 206_visit03_photo02.jpg name found
No Image 205_visit02_photo01.jpg name found
No Image 145_visit03_photo02.jpg name found
No Image 145_visit02_photo01.jpg name found
No Image 145_visit01_photo02.jpg name found
No Image 145_visit01_photo01.jpg name found
No Image 205_visit04_photo01.jpg name found
No Image 178_visit01_photo02.jpg name found
No Image 205_visit02_photo02.jpg name found
No Image 173_visit03_photo02.jpg name found
No Image 178_visit01_photo01.jpg name found
No Image 182_visit04_photo01.jpg name found
No Image 204_visit04_photo01.jpg name found
No Image 204_visit04_photo02.jpg name found
No Image 207_visit02_photo02.jpg name found
No Image 205_visit04_photo02.jpg name found
No Image 208_visit01_photo02.jpg name found
No Image 197_visit01_photo01.jpg name found
No Image 205_visit03_photo01.jpg name found
No Image 192_visit01_photo02.jpg name found
No Image 224_visit05_photo02.jpg name found
No Image 193_visit02_photo01.jpg name found
No Image 206_visit02_photo01.jpg